In [2]:
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
import cartopy
%matplotlib inline

def fix_calendar(ds, timevar='T'):
    if ds[timevar].attrs['calendar'] == '360':
        ds[timevar].attrs['calendar'] = '360_day'
    return ds

def fix_coords(ds):
    ds = ds.assign_coords(X=(((ds.X + 180) % 360) - 180)).sortby(['X','Y']).rename({'X':'lon','Y':'lat'})
    return ds

period= slice('2017-12-16','2018-11-16')
period1 = slice('2016-12-16','2017-11-16')